Basic Walkthrough
====================

Thoughts is a lightweight inference engine. This tutorial will walk you through how to use the engine, staring with a simple rule and working up to more complex rules.

## The Rules Engine

The RulesEngine class handles the assertion logic, context memory, and is the primary way of interacting with the engine. Start up a new instance with the following code.

In [1]:
# import the thoughts rules engine
import os, sys
sys.path.insert(1, os.path.abspath('..\\..'))
from thoughts.rules_engine import RulesEngine
 
engine = RulesEngine()

## Define Your Rules

The engine works by comparing incoming assertions (facts) to existing rules (the knowledge base, or KB). When it finds a matching rule, it will trigger that rule. This in turn can cause additional rules to trigger, and so on until no more rules are matched and no assertions are left to process.

This means we need to start by adding some rules to our engine. Let's start by adding a simple rule, to detect when the incoming assertion is the text "hello" and to echo back the text "hello, world".

To do this, we define a structure with a #when portion and a #then portion. This is the basic format for any rule that you define.

In [2]:
rule = {"#when": "hello",
        "#then": "hello, world!"}

## Load a Rule into the Engine

To add this rule to the engine, call the add_rule function.

In [3]:
engine.add_rule(rule)

# Asserting Facts

Now that we have added a rule, we can send assertions into the engine to have it process them. There are different ways to control this assertion cycle, but for now we'll keep it simple. The default behavior is for the engine to match all rules and keep the match-assert cycle going until no additional assertions are generated and there are no assertions left to process. The default behavior will also return the final set of assertions that were generated as a list.

In [4]:
result = engine.process("hello")
print(result)

['hello, world!']


## Loading Multiple Rules into the Engine

You can also add multiple rules to the engine at one time, by defining a list and calling the add_rules function.

In [5]:
rules = [{"#when": "thanks",
        "#then": "you're welcome!"},
        
        {"#when": "what is your name?",
        "#then": "my name is thought-bot"}]

# loads rules into memory directly from a list.
engine.add_rules(rules)

The engine now has three rules - the original "hello, world" rule, and now these two additional rules. Let's run another assertion to try it out.

In [6]:
result = engine.process("what is your name?")
print(result)

['my name is thought-bot']


## Loading Rules from a File

You can define your rules in a file and load them into the engine with a single command. This allows you to create rule files to use across projects or to share with others.

Here we'll get a little fancier with the result just for fun. Let's pretent we're implementing a chatbot and will return the first result generated by the engine.

In [7]:
engine.load_rules_from_file("data/sample_rules.json")

assertion = "how old are you?"
print("USER:", assertion)
result = engine.process(assertion)
print("BOT:", result[0])

assertion = "no"
print("USER:", assertion)
result = engine.process(assertion)
print("BOT:", result[0])

loaded 2 rules
USER: how old are you?
BOT: well now, that's a rather personal question, isn't it?
USER: no
BOT: you seem rather negative about this


## Using Wildcards

Rules are not limited to hardcoded text. You can place wildcard variables in the #when portion of your rules and the engine will attempt to "unify" or match these variables with the incoming assertion. Let's look at a simple example.

In [8]:
rule = {"#when": "my name is ?name",
        "#then": "hello, ?name!"}

engine.add_rule(rule)

When the engine is able to unify the incoming assertion with the rule's #when portion, it will remember the variables so that you can use them in the output.

In [9]:
result = engine.process("my name is jeremy")
print(result)

['hello, jeremy!']


## Matching on Facts

So far we've been using text examples for the assertions and rules. You can also match on assertions that are structures of information, commonly known as "facts".

A Fact is a collection of key/value pairs. Each value can either be a literal (string or number), a list, or another fact. In this way, you can combine facts into larger and larger structures of information.*

In the example below, the fact in the #when clause is {"temperature": "cold", "sky": "cloudy"}, representing the temperature on a cold and cloudy day. There's no special significance to the words "temperature" or "cloudy" to the engine. You as a rules developer are free to design these however you want.

*This is the same concept known in many programming languages as an "object". You can also think of it as a Python dict, or JSON.

In [10]:
rule = {"#when": {"temperature": "cold", "sky": "cloudy"},
        "#then": {"predict": "snow"}}

engine.add_rule(rule)

#### Example - Attribute Values Do Not Match

All attributes from the fact must be present in the incoming assertion for the rule to match, and all corresponding attributes must "unify" or match with the incoming assertion.

For example, the following assertion will not be matched, as the value in the temperature attribute does not match.

In [11]:
result = engine.process({"temperature": "hot", "sky": "cloudy"})
print(result)

[{'temperature': 'hot', 'sky': 'cloudy', '#seq-start': 0, '#seq-end': 1}]


#### Example - Attribute Values All Match

However, the following assertion will be matched, as the values in all attributes are the same.

In [12]:
result = engine.process({"temperature": "cold", "sky": "cloudy"})
print(result)

[{'predict': 'snow'}]


#### Example - Assertions with Extra Attributes Not in the Rule
Assertions that have extra attributes which are not in the rule's pattern will also match. The engine treats the extra attributes from the incoming assertion as extra information and effectively ignores them when performing the match.

In the example below, the attribute "wind" is effectively ignored. All of the other attributes, "temperate" and "sky" exist in the rule and with matching values and so the rule is triggered.

In [13]:
result = engine.process({"temperature": "cold", "sky": "cloudy", "wind": "SSW"})
print(result)

[{'predict': 'snow'}]


#### Example - Assertions with Missing Attributes

Finally, assertions that do not have all of the attributes which are present in the rule will not match. The assertion must have all of the attributes required of the rule. This allows you to create rules for slightly different but similiar situations.

In the example below, the assertion does not have the required "sky" attribute, and so the rule is not triggered.

In [14]:
result = engine.process({"temperature": "cold"})
print(result)

[{'temperature': 'cold', '#seq-start': 0, '#seq-end': 1}]


## Facts with Wilcards

Wildcards can also be used in the fact's structure. This can be a very powerful technique to carrying incoming information from the assertion and restructuring it into different structures in the triggered assertions.

In [15]:
rule = {"#when": {"player": "?player", "score": 100},
        "#then": {"wins": "?player"}}

engine.add_rule(rule)

result = engine.process({"player": "alice", "score": 100})
print(result)

[{'wins': 'alice'}]


## Multiple Actions

You can have more than one command (action) in the "then" portion

In [16]:
new_rule = { 
    "#when": "hi",
    "#then": [{"#output": "hello there"}, 
            {"#output": "nice to meet you"}]
}

engine.add_rule(new_rule)

result = engine.process("hi")

hello there
nice to meet you


## Storing Knowledge

You can store "item" knowledge (facts). Store the unique name of the item in the #item attribute. Then store additional information about that item with as many attributes as you need. You can also store complex information by storing lists and dictionary values within each attribute, as many levels as you need.

In [17]:
fact = {"#item": "user", "name": "jeremy", "dog": "hudson", "car": {"make": "jeep", "year": 2017}}
engine.add_rule(fact)

## Reference Stored Knowledge

You can reference the items (facts) and their properties in your rules, using $itemname.property syntax.

The $ prefix will tell the engine to find the matching #item with that name.

In [18]:
fact = {
        "#when": "what is my name", 
        "#then": [{"#output": "your name is $user.name"}]}
        
engine.add_rule(fact)

result = engine.process("what is my name")

your name is jeremy


## Tying it All Together - A Complete Example

Now let's see an example which ties together the concepts above.

The following is a weather prediction KB, with some basic natural language processing and a forward-chaining rule that stores the prediction in a variable and uses that stored information in a response.

In [19]:
rules = [
    
    # weather prediction rules
    { "#when": "cloudy", "#then": "rain is possible" },
    { "#when": "cold", "#then": "snow is possible" },

    # text matching rule
    { "#when": "it's ?condition today",
      "#then": "predict weather when ?condition"},

    # forward chain rule with knowledge store, retrieval, and output
    { "#when": "predict weather when ?condition",
      "#then": [
        { "#assert": "?condition", "#into": "$prediction" },
        { "#output": "that means $prediction" }]
    }
]

engine.clear_rules()
engine.add_rules(rules)

conclusions = engine.process("it's cold today")

that means {'#assert': 'snow is possible'}
